In [1]:


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import gmplot
import torch
from torch import nn
from tqdm import tqdm
from tqdm import trange
import time
from tqdm import trange
import h3
from collections import OrderedDict
from sklearn.preprocessing import  OneHotEncoder
import move_fromd2l as d2l

def geo_t_h3_norepeat(data):


        h3_list =OrderedDict()
        for i in data:
            a = h3.geo_to_h3(i[0], i[1], 10)
            # print(a)
            h3_list.setdefault(a)
        #这这里去掉h3的重复
        return h3_list



def h3_t_geo(data):
    new_list = []
    for i in data:
        i =h3.h3_to_geo(i)
        new_list.append(i)
    return new_list


def generate_h3_list(data,label='repeat'):
    if label=='repeat':
        alist = geo_t_h3_norepeat(data)
        # print(type(alist))
        LIST = list(alist.keys())
        return np.array(LIST)
    elif label=='no-repeat':
        LIST=[]
        for temp in data:
            LIST.append(h3.geo_to_h3(temp[0],temp[1],10))
        return np.array(LIST)
    else:
        return np.array([])

# draw(,130,10)
# list,文件名，放大率
def draw(list,number,b):
    new_list = torch.tensor(list)
    lat = []
    lng = []
    for i in new_list:
        lat.append(i[0])
        lng.append(i[1])
    lat = torch.stack(lat)
    lng =torch.stack(lng)
    gmap = gmplot.GoogleMapPlotter(lat[0], lng[0], b)
    gmap.plot(lat, lng,color='r',lw=10)  #描绘轨迹点
    gmap.draw("user{}.html".format(number))   #显示图
    print("over")
# 这个提取出来有5个维度
def dataset(user,start,end,step):
    # user:第几个用户
    # filenumber:取前面几天数据
    # step：隔了多少步取一次
    userdata =  '../Geolife Trajectories 1.3/Data/'+user+'/Trajectory/'
    filelist = os.listdir(userdata)  #返回指定路径下所有文件和文件夹的名字，并存放于一个列表中
    filelist.sort()
    names = ['lat','lng','zero','alt','days','date','time']
    df_list = [# f为文件索引号，header为列数，names为列表列名，index_col为行索引的列编号或列名
    pd.read_csv(userdata + f,header=6,names=names,index_col=False)
    for f in filelist[start:end]]
    df = pd.concat(df_list, ignore_index=True) #表格列字段不同的表合并
    df.drop(['zero', 'days'], axis=1, inplace=True) #drop函数默认删除行，列需要加axis = 1
    df_min = df.iloc[::step, :]
    return df_min
# 这个提取出来有2个维度


def synthetic_data(df_min):
    a =df_min['lat'].tolist()
    b = df_min['lng'].tolist()
    a = torch.tensor(a,dtype=torch.float,requires_grad=True).reshape((-1, 1))
    b = torch.tensor(b,dtype=torch.float,requires_grad=True).reshape((-1, 1))
    features = torch.concat([a,b],1)
    return features
# 将(lat,lng)变成哈希值





# 这个all_data_h3_list可以先shuffle一下






In [2]:
# 这个提取出来有5个维度
train_dataset = dataset("006",0,20,60)
test_dataset =  dataset("006",20,25,60)

# 这个提取出来有2个维度
train_data = synthetic_data(train_dataset)
test_data = synthetic_data(test_dataset)
# concat()
all_data =torch.concat([train_data,test_data],0)



Train_h3_list  = generate_h3_list(train_data,label='no-repeat')
TESETLIST  = generate_h3_list(test_data,label='no-repeat')
vocab = generate_h3_list(all_data,label='repeat')
print(len(Train_h3_list))
print(len(TESETLIST))
print(len(vocab))

encoder = OneHotEncoder(sparse=False).fit(vocab.reshape(-1,1))
def encoding(data):
    return encoder.transform(data.reshape(-1,1))
def decoding(one_hot_data):
    return encoder.inverse_transform(one_hot_data)




# 可以通过idnex找到对应的h3
index_h  = dict(enumerate(vocab))
# 可以通过h3编码找到对应的index
h_index ={h3:i for i ,h3 in index_h.items()}




def label_encode(data):
    return np.array([h_index[ch] for ch in data])

def label_decode(data):
    return np.array([index_h[ch] for ch in data])


#y是一个列表，k是步长
def dataloader(y,k):
    data =[]
    for i in range(len(y)-k+1):
        indata = y[i:i+k]
        outdata = y[i+k:i+k+1]
        data.append((indata,outdata))
    return data

train_dataloader = dataloader(Train_h3_list,10)
test_dataloader =dataloader(test_data,10)
# for i,j in train_dataloader:
#     print(i,j)


368
156
366


In [3]:
class RNN(nn.Module):
    def __init__(self,vocab) -> None:
        super().__init__()
        self.vocab = vocab
        self.model =nn.LSTM(
            input_size=vocab,
            hidden_size=320,
            batch_first=True,
            bias=True
        )

        self.output = nn.Sequential(
            nn.Linear(320,vocab)
            # nn.ReLU()
        )

    def forward(self,x):
        r_out,(h_out,c_out) = self.model(x.view(1,10,self.vocab) ,None)
        outdata = self.output(r_out[:,-1,:])
        return outdata
net = RNN(len(vocab)).to(d2l.try_gpu())
optimizer = torch.optim.SGD(net.parameters(),lr = 0.1,momentum=0.8)
optimizer_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_function = nn.CrossEntropyLoss(reduction='mean')
#这里无论是sum还是mean的结果是一样的




In [4]:
for epoch in trange(1000):
    loss_ =0
    optimizer_scheduler.step()
    optimizer.step()
    for i , k in train_dataloader[:-1]:
        optimizer.zero_grad()
        a = encoder.transform(i.reshape(-1,1))
        a =torch.tensor(a).to(torch.float32).to(d2l.try_gpu())
        k = label_encode(k)
        k =torch.tensor(k).to(torch.long).to(d2l.try_gpu())
        loss = loss_function(net(a),k)
        # print(loss)
        loss.backward()
        optimizer.step()
        loss_ = loss_+loss
    print(loss_)

  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\zhangliyu\.conda\envs\limuconda\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


tensor(5.8680, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9173, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8741, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8984, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8710, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9161, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8691, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8968, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9973, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9211, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9521, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8536, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9178, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8893, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9296, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8871, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9338, device='cuda:0', grad_fn=

  0%|          | 1/1000 [00:05<1:24:26,  5.07s/it]

tensor(4.4551, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0579, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2115.1543, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1352, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1753, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2342, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2580, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2263, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2730, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1084, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0403, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0980, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0012, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9191, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8490, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.3302, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.6062, device='cuda:0', grad_fn=<

  0%|          | 2/1000 [00:07<59:01,  3.55s/it]  

...

  0%|          | 5/1000 [00:14<43:24,  2.62s/it]

tensor(4.6020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0644, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.1020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.5610, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.1351, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.6523, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.8828, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0427, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.9257, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7858, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.0496, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5682, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.4786, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(10.0961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.2892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.1188, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.6713, device='cuda:0', grad_fn

  0%|          | 5/1000 [00:16<55:53,  3.37s/it]

tensor(3.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.3577, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.4360, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.1670, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.8780, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7441, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.4370, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7137, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.2807, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.9378, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.4689, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.1091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.5365, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.2278, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.8267, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.2954, device='cuda:0', grad_fn=

KeyboardInterrupt: 